<div align="center">
  <img src='https://i.ibb.co/d064HCdW/androidstudio-icon-md.png' height="75" alt="Android Studio Logo"/>
  
  [![GitHub](https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white)](https://github.com/mrtear/Extension-Source-Builder)
</div>

🚨 Note:
- *Runtime Duration (FREE):*
   - TPU Runtime: ~3h 20m | ~334GB RAM, 96vCPU
   - CPU Runtime: ~85h 40m | ~12GB RAM, 2vCPU

- *Before Closing:*
   - Properly terminate session: `Runtime → Disconnect and delete runtime`

##### *Make sure to run each cell in order.*

In [ ]:
#@title 🔧 Install Dependencies & Packages
import os
import re
import urllib.request

# Suppress intermediate output and install essential packages
!apt-get update -qq > /dev/null 2>&1 && apt-get install -qq -y openjdk-17-jdk-headless android-sdk gradle > /dev/null 2>&1

# Set the main SDK directory
destination = "/lib/android-sdk"

# Fetch the latest command-line tools URL for Linux
url = "https://developer.android.com/studio"
html = urllib.request.urlopen(url).read().decode('utf-8')
match = re.search(r"https:\/\/dl\.google\.com\/android\/repository\/commandlinetools-linux-[0-9]*_latest\.zip", html)

if match:
    tools_download_url = match.group(0)

    # Download the command-line tools zip quietly
    !curl --location -o /content/android.zip {tools_download_url} > /dev/null 2>&1

    # Create & Unpack the zip into the temporary directory /content quietly
    !mkdir -p /content/android-temp
    !unzip -q /content/android.zip -d /content/android-temp

    # Create the necessary directory structure
    !mkdir -p "{destination}/cmdline-tools/latest"

    # Move the contents to the correct location, avoiding nested directories
    !mv /content/android-temp/cmdline-tools/* "{destination}/cmdline-tools/latest" > /dev/null 2>&1

    # Clean up the temporary files
    !rm -rf /content/android-temp /content/android.zip /usr/lib/android-sdk/build-tools/debian > /dev/null 2>&1

    # sdkmanager configuration
    !yes | /lib/android-sdk/cmdline-tools/latest/bin/sdkmanager --licenses > /dev/null 2>&1
    !/lib/android-sdk/cmdline-tools/latest/bin/sdkmanager "platforms;android-34" "build-tools;34.0.0" "platform-tools" > /dev/null 2>&1

    # Set environment variables
    os.environ['ANDROID_HOME'] = destination
    os.environ['PATH'] += f":{destination}/cmdline-tools/latest/bin"

    # Print completion message
    print("🛠️ You're good to go! Clone your repo next!")
else:
    print("Failed to fetch the tools_download_url")

In [ ]:
# @title ### 📥 Clone and **Clean** Repository
# @markdown ### Enter your GitHub `username`:
username = "" # @param {type:"string"}
# @markdown ### Enter your project name `repo name`:
project_name = "extensions-source" # @param {type:"string"}
# @markdown ### Enter branch name `leave empty for default/main`:
branch_name = "" # @param {type:"string"}
# @markdown ### `Optional Cleaning` Clone only a specific language folder (leave empty to clone everything):
languages = "" # @param ["", "ar", "bg", "ca", "cs", "de", "en", "es", "fr", "id", "it", "ja", "ko", "pl", "pt", "ru", "th", "tr", "uk", "vi", "zh"] {type:"string"}
# @markdown ### `Optional Cleaning` Enter source names to keep and remove others in the selected language (separated by commas, e.g., `hehescans,9scans`):
sources = "" # @param {type:"string"}

import os
import subprocess
import shutil
from typing import Optional

def clone_repository(username: str, project_name: str, branch_name: Optional[str] = None) -> bool:
    """
    Clone or update a GitHub repository with optional branch selection.
    Args:
        username (str): GitHub username
        project_name (str): Repository name
        branch_name (Optional[str]): Branch name to clone (default/main if None)
    Returns:
        bool: True if operation succeeded, False otherwise
    """
    if not username:
        print("❌ Error: Username not provided.")
        return False

    if not project_name:
        print("❌ Error: Project name not provided.")
        return False

    repo_url = f"https://github.com/{username}/{project_name}.git"
    target_dir = "/content/extensions-source"
    try:
        # Clean up existing directory if it exists
        if os.path.exists(target_dir):
            print("🧹 Cleaning up existing repository...")
            subprocess.run(["rm", "-rf", target_dir], capture_output=True)
        print("📥 Cloning fresh repository...")
        # Ensure we're in /content directory first
        os.chdir("/content")
        # Create target directory
        os.makedirs(target_dir, exist_ok=True)
        # Change to target directory before cloning
        os.chdir(target_dir)
        # Prepare git command
        git_cmd = ["git", "clone"]
        # Add branch flag if specified
        if branch_name:
            git_cmd.extend(["-b", branch_name])
        # Complete clone command
        git_cmd.extend([repo_url, "."])
        # Execute clone command
        result = subprocess.run(git_cmd, capture_output=True, text=True)
        if result.returncode == 0:
            # Create the local.properties file with the SDK path
            with open("local.properties", "w") as f:
                f.write("sdk.dir=/lib/android-sdk")
            # Set up JVM arguments
            performance_settings = "org.gradle.jvmargs=-Xmx32g -Dfile.encoding=UTF-8"
            with open("gradle.properties", "a") as gradle_file:
                gradle_file.write(performance_settings)
            print("✅ Repository cloned successfully!")
            return True
        else:
            raise Exception(f"Clone failed: {result.stderr}")
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

def delete_lang_folders():
    """Delete all language folders except selected ones"""
    base_path = "/content/extensions-source/src"
    folders = os.listdir(base_path)

    # Split comma-separated input into list
    selected_list = [f.strip() for f in languages.split(",")]

    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path) and folder not in selected_list:
            shutil.rmtree(folder_path)

    print(f"Retained folders: {', '.join(selected_list)}")

def delete_source_folders():
    """Delete folders except selected ones inside selected language folder"""
    base_path = "/content/extensions-source/src"
    selected_lang = languages
    source_names = [s.strip() for s in sources.split(",")]  # Split the comma-separated list

    if selected_lang and not sources:
        print(f"Language selected but no sources provided. Keeping all files")
        return

    if sources and not selected_lang:
        print(f"Sources provided but no language selected. Keeping all files")
        return

    if selected_lang:  # Only process if a language is specified
        lang_path = os.path.join(base_path, selected_lang)
        if not os.path.exists(lang_path):
            print(f"Language folder '{selected_lang}' not found!")
            return

        # Delete folders not specified inside the selected language folder
        for folder in os.listdir(lang_path):
            folder_path = os.path.join(lang_path, folder)
            if os.path.isdir(folder_path) and folder not in source_names:
                shutil.rmtree(folder_path)

        # Delete all other language directories
        for lang in os.listdir(base_path):
            lang_path = os.path.join(base_path, lang)
            if os.path.isdir(lang_path) and lang != selected_lang:
                shutil.rmtree(lang_path)

        print(f"Only kept: {selected_lang}/{', '.join(source_names)}")

# Execute the clone operation
success = clone_repository(username, project_name, branch_name)
if success:
    if languages:
        if sources:
            delete_source_folders()
        else:
            delete_lang_folders()
    else:
        print("🌟 Cloning complete without any specific language cleaning.")
else:
    print("❌ Operation failed during cloning.")

In [ ]:
# @title <img src='https://i.ibb.co/Z6kXZ1mc/androidstudio-1024x1024.png' height="20" /></a> Build Extension
# @markdown ### Select a source language:
lang = "" # @param ["", "all", "ar", "bg", "ca", "cs", "de", "en", "es", "fr", "id", "it", "ja", "ko", "pl", "pt", "ru", "th", "tr", "uk", "vi", "zh"]
# @markdown ### Enter the source name (e.g., `hehescans`):
source = "" # @param {type:"string"}
# @markdown ### Show full build output?:
show_output = False # @param {type:"boolean"}

import subprocess
import os

# Build the specific extension using the provided lang and source
build_command = f"./gradlew :src:{lang}:{source}:assembleDebug --console=plain"
result = subprocess.run(build_command, shell=True, capture_output=True, text=True)

# Print the full build output if requested
if show_output:
    print("\nFull Build Output:\n")
    print(result.stdout)
    print(result.stderr)

# Check if the build was successful
build_success = result.returncode == 0

# Print the initial message based on build success
if build_success:
    print("🎉 Build succeeded! Your extension is ready.")
    print(f"Check the APK at: /content/extensions-source/src/{lang}/{source}/build/outputs/apk/debug")
else:
    print("❌ Build failed. Please check the logs for details or check show_output box to see build errors again.")

In [ ]:
# @title ### 📤 Upload APK  and Generate QR Code to Download
# @markdown ### Select a source language:
lang = "" # @param ["", "all", "ar", "bg", "ca", "cs", "de", "en", "es", "fr", "id", "it", "ja", "ko", "pl", "pt", "ru", "th", "tr", "uk", "vi", "zh"]
# @markdown ### Enter the source name (e.g., `hehescans`):
source = "" # @param {type:"string"}

# Install necessary libraries
!pip install -q requests pyqrcode pypng > /dev/null 2>&1

import os
import glob
import requests
import pyqrcode
from IPython.display import clear_output, Image, display

clear_output()

# Construct the APK directory path
apk_dir_path = f"/content/extensions-source/src/{lang}/{source}/build/outputs/apk/debug/"

# Search for the APK file in the specified directory
apk_files = glob.glob(os.path.join(apk_dir_path, "*.apk"))

if apk_files:
    # Use the first APK file found
    apk_path = apk_files[0]
    print(f"✅Found APK Path: {apk_path}")

    # Upload to Catbox
    url = "https://litterbox.catbox.moe/resources/internals/api.php"
    files = {
        'fileToUpload': (os.path.basename(apk_path), open(apk_path, 'rb')),
        'time': (None, '1h'),
        'reqtype': (None, 'fileupload')
    }
    response = requests.post(url, files=files)

    if response.status_code == 200:
        dl_url = response.text

        # Generate and display QR code
        qr = pyqrcode.create(dl_url)
        img = "myqr.png"
        qr.png(img, scale=10)
        display(Image(img))
        print("Download URL:", dl_url)
    else:
        print("❌Error uploading the APK. Please try again.")
else:
    print(f"❌Error: No APK files found in the directory: {apk_dir_path}")